In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import numpy as np
import json
import pandas as pd
import numpy as np
import time
import string
from matplotlib import pyplot as plt
import tensorflow as tf
import sys
sys.path.append('../')
from brain2brain import utils
from brain2brain import generators
%matplotlib inline

# TF
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model

# TCN
from brain2brain.tcn import TCN
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, GRU

In [3]:
# Read saved paths for training.
saved_paths = utils.get_file_paths("../brain2brain/train_676_norm_files_projects.txt")

In [4]:
# Split the train files into a training and validation set.
train, val = utils.split_file_paths(saved_paths, 0.8)

In [5]:
# The time we look back.
lookback_window = 256 # .5 seconds
# Length of sequence predicted.
length_pred = 1 # 1 timestep
# Delay between lookback and length.
delay_pred = 0 # No delay.
# Sampling of electrodes.
samples_per_second = 1 # Samples Per Second
timesteps_per_sample = int(lookback_window // samples_per_second)
# Electrodes
electrode_selection = [1]
electrode_count = len(electrode_selection)
batch_size = 512

In [6]:
# Training Generator
train_generator = generators.FGenerator(file_paths = train,
                                        lookback=lookback_window, length = length_pred, delay = delay_pred,
                                        batch_size = batch_size, sample_period = samples_per_second,
                                        electrodes= electrode_selection, shuffle = True, debug=False,
                                        ratio=1.0)

In [7]:
# Validation Generator
val_generator = generators.FGenerator(file_paths = val,
                                      lookback=lookback_window, length = length_pred, delay = delay_pred,
                                      batch_size = batch_size, sample_period = samples_per_second,
                                      electrodes= electrode_selection, shuffle = False, debug=False,
                                      ratio=1.0)

In [8]:
len(train_generator)

304

In [9]:

len(val_generator)

102

In [ ]:
model = Sequential()
model.add(GRU(32,
              dropout=0.2,
              recurrent_dropout=0.2,
              input_shape=(timesteps_per_sample, electrode_count)))
model.add(Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
model.summary()
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data = val_generator)